In [12]:
import xml.etree.ElementTree as ET
from collections import defaultdict
from zipfile import *
from docx import Document
from docx.shared import Inches
from docx.shared import Pt
import re

filename = '/home/abelalina/Desktop/descriptor/Тест.zip'
#Берём на вход zip-архив и извлекаем из него файлы
z = ZipFile(filename)
z.extractall()

#Парсим cas_config.xml
cas_tree = ET.parse('cas_config.xml')
cas_root = cas_tree.getroot()
#Парсим pas_config.xml
pas_tree = ET.parse('pas_config.xml')
pas_root = pas_tree.getroot()

#Функция, которая извлекает всю информацию о технологиях (не категориях!) из файла cas_config.xml
#(ту, что находится в секциях Def: id элемента технологии, название, условия срабатывания)
def GetTechnologies(root):
    technologies = {}
    for element in root:
        if element.tag == 'TextObjectDef':
            technologie_id = element.attrib['id']
            technologie_name = element.attrib['name']
            technologies[technologie_id] = technologie_name
        elif element.tag == 'FingerprintDef':
            technologie_id = element.attrib['id']
            technologie_name = element.attrib['name']
            technologie_type = element.attrib['type']
            if technologie_type == 'fingerprint':
                technologie_text_value_threshold = element.attrib['text_value_threshold']
                info = (technologie_name, technologie_type, technologie_text_value_threshold)
            else:
                info = (technologie_name, technologie_type)
            technologies[technologie_id] = info
        elif element.tag == 'FormEtalonDef':
            technologie_id = element.attrib['id']
            technologie_name = element.attrib['name']
            conditions = []
            for condition in element:
                if condition.tag == 'Condition':
                    info = {}
                    condition_id = condition.attrib['id']
                    condition_value = condition.attrib['value']
                    condition_min_rows = condition.attrib['min_rows']
                    info[condition_id] = (condition_value, condition_min_rows)
                    conditions.append(info)
            technologies[technologie_id] = (technologie_name, conditions)
        elif element.tag == 'TableEtalonDef':
            technologie_id = element.attrib['id']
            technologie_name = element.attrib['name']
            conditions = []
            for condition in element:
                if condition.tag == 'Condition':
                    info = {}
                    condition_id = condition.attrib['id']
                    condition_value = condition.attrib['value']
                    condition_min_rows = condition.attrib['min_rows']
                    info[condition_id] = (condition_value, condition_min_rows)
                    conditions.append(info)
            technologies[technologie_id] = (technologie_name, conditions)
    return technologies

#Функция, которая извлекает информацию о категориях из файла cas_config.xml: id, название, тип категории)
def GetCategories(root, categories = {}):
    for element in root:
        if element.tag == 'Category':
            cat_id = element.attrib['id']
            cat_name = element.attrib['name']
            cat_type = element.attrib['type']
            categories[cat_id] = (cat_name, cat_type)
            GetCategories(element, categories)
    return categories

#Задаём словарь на основе файла pas_config.xml, где ключи - id объектов защиты, а значения - их названия
pd_dic = {}
for elem in pas_root:
    if elem.tag == 'ProtectedDocumentDef':
        pd_dic[elem.attrib['id']] = elem.attrib['name']

#Функция, которая на основе файла pas_config.xml строит "дерево" каталогов ОЗ и самих ОЗ
def Catalogs (root, name, tree=None):
    if tree is None:
        tree = defaultdict(list)
    for elem in root:
        if elem.tag == 'ProtectedDocument':
            doc_id = elem.attrib['id']
            tree[name].append((pd_dic[doc_id], 'document'))
        elif elem.tag == 'ProtectedCatalog':
            cat_name = (elem.attrib['name'], 'catalog')
            if cat_name not in tree[name]:
                tree[name].append(cat_name)
                Catalogs(elem, cat_name, tree)
            else:
                assert False, 'дубликат'
    return tree

#Функция, которая на основе файла pas_config.xml и функций Catalogs и GetTechnologies строит словарь,
#где ключи - название ОЗ, а значения - название ЭНТ и условие его срабатывания
def Protected_objects(root):
    protected_objects_info = {}
    for elem in root:
        if elem.tag == 'ProtectedCatalog':
            catalog_name = elem.attrib['name']
            catalogs_dict = Catalogs(elem, catalog_name)
        if elem.tag == 'ProtectedDocumentDef':
            doc_name = elem.attrib['name']
            conditions = []
            for condition in elem:
                if condition.tag == 'Condition':
                    condition_info = []
                    for entry in condition:
                        entry_type = entry.attrib['entry_type']
                        entry_id = entry.attrib['entry_id']
                        if entry_type == 'text_object':
                            quantity_threshold = entry.attrib['quantity_threshold']
                            entry_name = technologies[entry_id]
                            entry_info = (entry_name, entry_type, quantity_threshold)
                        elif entry_type == 'form':
                            form_etalon_condition_id = entry.attrib['etalon_condition_id']
                            entry_name = technologies[entry_id][0]
                            form_conditions = technologies[entry_id][1]
                            for form_condition in form_conditions:
                                for key in form_condition:
                                    if key == form_etalon_condition_id:
                                        value = form_condition[key][0]
                                        min_rows = form_condition[key][1]
                            entry_info = (entry_name, entry_type, value, min_rows)
                        elif entry_type == 'table':
                            table_etalon_condition_id = entry.attrib['etalon_condition_id']
                            entry_name = technologies[entry_id][0]
                            table_conditions = technologies[entry_id][1]
                            for table_condition in table_conditions:
                                for key in table_condition:
                                    if key == table_etalon_condition_id:
                                        table_etalon_condition = table_condition[key][0]
                                        number_of_rows = table_condition[key][1]
                            entry_info = (entry_name, entry_type, table_etalon_condition, number_of_rows)
                        elif entry_type == 'category':
                            entry_name = categories[entry_id][0]
                            cat_type = categories[entry_id][1]
                            entry_info = (entry_name, (entry_type, cat_type))
                        else:
                            entry_name = technologies[entry_id][0]
                            entry_type = technologies[entry_id][1]
                            if entry_type == 'fingerprint':
                                entry_info = (entry_name, entry_type, technologies[entry_id][2])
                            else:
                                entry_info = (entry_name, entry_type)
                        condition_info.append(entry_info)
                    conditions.append(condition_info)
                protected_objects_info[doc_name] = conditions

    return protected_objects_info

#Функция, которая на вход принимает список названий ЭНТ и их типов, входящих в каждый ОЗ - итеративно,
#а возвращает списки кортежей: наименование технологии - название ЭНТ. Например: ('Категории БКФ', "Маркетинг")
#Функция нужна для формирования столбца "Элементы настройки технологий"
def UsedTechnologies(arr):
    technologies = []
    categories = []
    text_objects = []
    fingerprints = []
    forms = []
    tables = []
    stamps = []
    graph_objects = []
    autoling = []
    tech_catalogs = []

    for array in arr:
        for element in array:
            if type(element[1]) is tuple:
                if element[1][1] == 'term':
                    if element[0] not in categories:
                        categories.append(element[0])
                else:
                    if element[0] not in tech_catalogs:
                        tech_catalogs.append((element[0], element[1][1]))
            elif element[1] == 'text_object':
                if element[0] not in text_objects:
                    text_objects.append(element[0])
            elif element[1] == 'fingerprint':
                if element[0] not in fingerprints:
                    fingerprints.append(element[0])
            elif element[1] == 'form':
                if element[0] not in forms:
                    forms.append(element[0])
            elif element[1] == 'table':
                if element[0] not in tables:
                    tables.append(element[0])
            elif element[1] == 'stamp':
                if element[0] not in stamps:
                    stamps.append(element[0])
            elif element[1] == 'card':
                if element[0] not in graph_objects:
                    graph_objects.append(element[0])
            elif element[1] == 'image_classifier':
                if element[0] not in graph_objects:
                    graph_objects.append(element[0])
            elif element[1] == 'autoling':
                if element[0] not in autoling:
                    autoling.append(element[0])

    if len(categories)>0:
        pair = ('Категории БКФ', categories)
        technologies.append(pair)
    if len(text_objects)>0:
        pair = ('Текстовые объекты', text_objects)
        technologies.append(pair)
    if len(fingerprints)>0:
        pair = ('Эталонные документы', fingerprints)
        technologies.append(pair)
    if len(forms)>0:
        pair = ('Бланки', forms)
        technologies.append(pair)
    if len(tables)>0:
        pair = ('Выгрузки из баз данных', tables)
        technologies.append(pair)
    if len(stamps)>0:
        pair = ('Печати', stamps)
        technologies.append(pair)
    if len(graph_objects)>0:
        pair = ('Графические объекты', graph_objects)
        technologies.append(pair)
    if len(autoling)>0:
        pair = ('Категории Автолингвиста', autoling)
        technologies.append(pair)
    if len(tech_catalogs)>0:
        for tech in tech_catalogs:
            if tech[1] == 'text_object':
                pair = ('Каталоги Текстовых объектов', [tech[0]])
                technologies.append(pair)
            elif tech[1] == 'fingerprint':
                pair = ('Каталоги Эталонных документов', [tech[0]])
                technologies.append(pair)
            elif tech[1] == 'form':
                pair = ('Каталоги Бланков', [tech[0]])
                technologies.append(pair)
            elif tech[1] == 'table':
                pair = ('Каталоги Выгрузок из БД', [tech[0]])
                technologies.append(pair)
            elif tech[1] == 'stamp':
                pair = ('Каталоги Печатей', [tech[0]])
                technologies.append(pair)
            elif tech[1] == 'image_classifier':
                pair = ('Категории Графических объектов', [tech[0]])
                technologies.append(pair)
            elif tech[1] == 'autoling':
                pair = ('Категории Автолингвиста', [tech[0]])
                technologies.append(pair)

    return technologies

#Функция, которая принимает на вход списки, которые сформировала функция Protected_objects,
#и формирует описание условия срабатывания для столбца "Условия обнаружения объектов защиты"
def Condition(arr):
    elements = []
    for element in arr:
        if type(element[1]) is tuple:
            if element[1][1] == 'term':
                elements.append('Категория БКФ "' + element[0] + '"')
            else:
                if element[1][1] == 'text_object':
                    elements.append('Каталог ТО "' + element[0] + '"')
                elif element[1][1] == 'fingerprint':
                    elements.append('Каталог ЭД "' + element[0] + '"')
                elif element[1][1] == 'form':
                    elements.append('Каталог Бланков "' + element[0] + '"')
                elif element[1][1] == 'table':
                    elements.append('Каталог Выгрузок из БД "' + element[0] + '"')
                elif element[1][1] == 'stamp':
                    elements.append('Каталог Печатей "' + element[0] + '"')
                elif element[1][1] == 'image_classifier':
                    elements.append('Категория ГО "' + element[0] + '"')
                elif element[1][1] == 'autoling':
                    elements.append('Категория Автолингвиста "' + element[0] + '"')
        elif element[1] == 'text_object':
            elements.append('Текстовый объект "' + element[0] + '" (порог срабатывания = ' + element[2] + ')')
        elif element[1] == 'fingerprint':
            elements.append('Эталонный документ "' + element[0] + '" (порог цитируемости = ' + element[2] + ')')
        elif element[1] == 'form':
            elements.append('Эталонный бланк "' + element[0] + '" (минимальное количество совпадающих полей = ' + str(element[3]) + ', процент цитируемости = ' + str(element[2]) + ')')
        elif element[1] == 'table':
            elements.append('Выгрузка из БД "' + element[0] + '" (условие детектирования: ' + str(element[2]) + ', минимальное количество перехваченных строк = ' + str(element[3]) + ')')
        elif element[1] == 'stamp':
            elements.append('Эталонная печать "' + element[0] + '"')
        elif element[1] == 'card':
            elements.append('Графический объект "' + element[0] + '"')
        elif element[1] == 'image_classifier':
            elements.append('Графический объект "' + element[0] + '"')
        elif element[1] == 'autoling':
            elements.append('Категория Автолингвиста "' + element[0] + '"')
    condition = ' И '.join(elements)
    return condition

#Функция, которая аккумулирует результаты работы функции Condition
def DetectionConditions(arr):
    text = 'ОЗ срабатывает, если срабатывает '
    conditions = []
    for element in arr:
        conditions.append(Condition(element))
    text = text + '\rИЛИ\r'.join(conditions)
    return text

technologies = GetTechnologies(cas_root)
categories = GetCategories(cas_root)
catalogs = Catalogs(pas_root, None)
objects = Protected_objects(pas_root)

#Создаём документ docx
document = Document()
#Добавляем заголовок, в него входят слова "Описание ОЗ "
#и название файла - полный путь к файлу, за исключением расширения и директорий
document.add_heading('Описание ОЗ ' + str(re.sub(r'[A-z0-9/]{1,50}/', '', filename)[:-4]), 0)

#В документе создаём таблицу пока с одной строкой и с нулем столбцов
#(они потом добавятся по ходу исполнения функций ниже)
#Также здесь описывается оформление таблицы
table = document.add_table(rows=1, cols=0)
table.style = 'Light Grid Accent 1'
style = document.styles['Normal']
font = style.font
font.name = 'Times New Roman'
font.size = Pt(12)

#Функция, которая записывает в первые N столбцов каталоги ОЗ и сами ОЗ
def WriteCatalogs(root, row=0, column=0):
#Задаём нуль как первоначально количество строк
    num_rows_taken = 0
#Сортируем дерево каталогов ОЗ
#По мере обращения к каждому каталогу ОЗ/самому ОЗ добавляем новые строки и столбцы
    for catalog in sorted(catalogs[root]):
        while row + 1 > len(table.rows):
            table.add_row()
        while column + 1 > len(table.columns):
            table.add_column(Inches(1))
        if catalog[1] == 'catalog':
            table.rows[row].cells[column].text = catalog[0]
            rows = max(1, WriteCatalogs(catalog, row, column + 1))
            row += rows
            num_rows_taken += rows
        elif catalog[1] == 'document':
            table.rows[row].cells[-1].text = catalog[0]
            rows = max(1, WriteCatalogs(catalog, row, column + 1))
            row += rows
            num_rows_taken += rows
    return num_rows_taken

#Функция, которая заполняет последние два столбца таблицы
def WritePD():
#Для начала вызываем функцию WriteCatalogs, затем добавляем два столбца
#(второй столбец шире, т.к. в нём, как правило, больше текста)
    WriteCatalogs(None, 1, 0)
    table.add_column(Inches(1))
    table.add_column(Inches(2))
#Обращаемся к данным каждой строки третьего с конца столбца (там записаны объекты защиты)
#Если ОЗ есть в результате работы функции Protected_objects,
#то записываем в последние два столбца ЭНТ и условия их срабатывания
    for i in range(len(table.rows)):
        if table.rows[i].cells[-3].text in objects:
            techs = UsedTechnologies(objects[table.rows[i].cells[-3].text])
            table.rows[i].cells[-2].text = techs[0][0]
            for tech in techs[0][1]:
                table.rows[i].cells[-2].add_paragraph(tech, style='List Bullet')
            for ind in range(1, len(techs)):
                table.rows[i].cells[-2].add_paragraph(techs[ind][0], style='Body Text')
                for tech in techs[ind][1]:
                    table.rows[i].cells[-2].add_paragraph(tech, style='List Bullet')
            
            table.rows[i].cells[-1].text = DetectionConditions(objects[table.rows[i].cells[-3].text])
        else:
            table.rows[i].cells[-2].text = ''
            table.rows[i].cells[-1].text = ''
#Задаём наименования столбцов таблицы           
    hdr_cells = table.rows[0].cells
    for i in range(len(table.columns)-3):
        hdr_cells[i].text = 'Каталог ОЗ ' + str(i+1)
    hdr_cells[-3].text = 'Объекты защиты'
    hdr_cells[-2].text = 'Элементы настройки технологий'
    hdr_cells[-1].text = 'Условия обнаружения объектов защиты'

WritePD()
document.add_page_break()
document.save('ОЗ ' + str(re.sub(r'[A-z0-9/]{1,50}/', '', filename)[:-4])+ '_описание.docx')